# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:

* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

In [3]:
# GraphLab
import graphlab

## Load House Sale Data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [4]:
sales = graphlab.SFrame("kc_house_data.gl/")

## Useful Function

In [5]:
# Library
import math
import random
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [6]:
def print_coefficients(model):    
    # Get the degree of the polynomial
    deg = len(model.coefficients["value"]) - 1

    # Get learned parameters as a list
    w = list(model.coefficients["value"])

    # Numpy has a nifty function to print out polynomials in a pretty way
    # (We'll use it, but it needs the parameters in the reverse order)
    print "Learned polynomial for degree " + str(deg) + ":"
    w.reverse()
    print np.poly1d(w)

## Create New Feature

As in Week 2, we consider features that are some transformations of inputs.

In [8]:
from math import log, sqrt
sales["sqft_living_sqrt"] = sales["sqft_living"].apply(sqrt)
sales["sqft_lot_sqrt"] = sales["sqft_lot"].apply(sqrt)
sales["bedrooms_square"] = sales["bedrooms"] * sales["bedrooms"]

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales["floors"] = sales["floors"].astype(float) 
sales["floors_square"] = sales["floors"] * sales["floors"]

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of `sqft_living` will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

## Learn Regression Weight, L1 Penalty

Let us fit a model with all the features available, plus the features we just created above.

In [9]:
all_features = ["bedrooms",
                "bedrooms_square",
                "bathrooms",
                "sqft_living",
                "sqft_living_sqrt",
                "sqft_lot",
                "sqft_lot_sqrt",
                "floors",
                "floors_square",
                "waterfront",
                "view",
                "condition",
                "grade",
                "sqft_above",
                "sqft_basement",
                "yr_built",
                "yr_renovated"]

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty = 0` to ensure we don't introduce an additional L2 penalty.

In [10]:
model_all = graphlab.linear_regression.create(sales,
                                              target = "price",
                                              features = all_features,
                                              validation_set = None, 
                                              l2_penalty = 0.,
                                              l1_penalty = 1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.353822     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.375347     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [11]:
print_coefficients(model_all)

model_all.get("coefficients").print_rows(num_rows = 20)

Learned polynomial for degree 17:
       14         13         5         4        3
20.02 x  + 842.1 x  + 350.1 x + 24.42 x + 8469 x + 2.749e+05
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      | 

In [12]:
# Check zero and non-zero weight
zero_weight_count = 0
for value in model_all.coefficients["value"]:
    if value == 0:
        zero_weight_count = zero_weight_count + 1

#print "Non-Zero Weight Count: {0}".format(len(model_all.coefficients["value"]) - zero_weight_count)

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen?

## Select L1 Penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:

* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use `seed = 1` to ensure you get the same answer!

In [12]:
(training_and_validation, testing) = sales.random_split(.9, seed = 1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed = 1) # split training into train and validate

Next, we write a loop that does the following:

* for `l1_penalty` `in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7]` (to get this in Python, type `np.logspace(1, 7, num = 13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty = l1_penalty` and `l2_penalty = 0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [13]:
lowest_rss = None
best_l1_penalty = None

l1_penalty_value = np.logspace(1, 7, num = 13)

for l1_penalty in l1_penalty_value:
    print "L1 Penalty: {0}".format(l1_penalty)
    model_l1 = graphlab.linear_regression.create(training,
                                                 target = "price",
                                                 features = all_features,
                                                 l2_penalty = 0.,
                                                 l1_penalty = l1_penalty,
                                                 validation_set = None,
                                                 verbose = False)
    # Prediction and RSS
    prediction = model_l1.predict(validation)
    residual = prediction - validation["price"]
    residual_square = residual ** 2
    rss = residual_square.sum()
    
    # Check lowest RSS
    if lowest_rss is None or rss < lowest_rss:
        lowest_rss = rss
        best_l1_penalty = l1_penalty
    
    print "RSS: {0}".format(rss)

L1 Penalty: 10.0
RSS: 6.25766285142e+14
L1 Penalty: 31.6227766017
RSS: 6.25766285362e+14
L1 Penalty: 100.0
RSS: 6.25766286058e+14
L1 Penalty: 316.227766017
RSS: 6.25766288257e+14
L1 Penalty: 1000.0
RSS: 6.25766295212e+14
L1 Penalty: 3162.27766017
RSS: 6.25766317206e+14
L1 Penalty: 10000.0
RSS: 6.25766386761e+14
L1 Penalty: 31622.7766017
RSS: 6.25766606749e+14
L1 Penalty: 100000.0
RSS: 6.25767302792e+14
L1 Penalty: 316227.766017
RSS: 6.25769507644e+14
L1 Penalty: 1000000.0
RSS: 6.25776517727e+14
L1 Penalty: 3162277.66017
RSS: 6.25799062845e+14
L1 Penalty: 10000000.0
RSS: 6.25883719085e+14


*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [14]:
print "Lowest RSS: {0}".format(lowest_rss)
print "Best L1 Penalty: {0}".format(best_l1_penalty)

Lowest RSS: 6.25766285142e+14
Best L1 Penalty: 10.0


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [15]:
print_coefficients(model_l1)

model_l1.get("coefficients").print_rows(num_rows = 20)

Learned polynomial for degree 17:
       17         16       15         14        13        12
55.33 x  + 9.395 x  + 122 x  + 43.26 x  + 6200 x  + 6585 x 
              11             10             9             8         7
 + 9.309e+04 x  + 5.985e+05 x  + 1.287e+04 x + 2.115e+04 x + 147.9 x
             6        5         4             3         2
 - 0.000824 x + 1124 x + 39.11 x + 2.539e+04 x + 928.1 x + 7923 x + 1.96e+04
+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None |   19604.7163508    |  None  |
|     bedrooms     |  None |   7922.62023075    |  None  |
| bedrooms_square  |  None |   928.112147889    |  None  |
|    bathrooms     |  None |    25392.743443    |  None  |
|   sqft_living    |  None |   39.1083048767    |  None  |
| sqft_living_sqrt |  None |   1123.95233925    |  None  |
|     sqft_lot     |  None | -0.00082398

## Limit Non-Zero Weight

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:

1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [16]:
max_nonzeros = 7

## Explore Larger Range Value To Find Narrow Range With Desire Sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [17]:
l1_penalty_values = np.logspace(8, 10, num = 20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* for `l1_penalty` in `np.logspace(8, 10, num = 20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty = l1_penalty` and `l2_penalty = 0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [18]:
l1_penalty_min = None
l1_penalty_max = None

l1_penalty_min_list = []
l1_penalty_max_list = []

# Loop
for l1_penalty in l1_penalty_values:
    print "L1 Penalty: {0}".format(l1_penalty)
    model_l1_limit = graphlab.linear_regression.create(training,
                                                       target = "price",
                                                       features = all_features,
                                                       l2_penalty = 0.,
                                                       l1_penalty = l1_penalty,
                                                       validation_set = None,
                                                       verbose = False)
    
    model_l1_non_zero = model_l1_limit["coefficients"]["value"].nnz() - 1
    
    # Check for L1 penalty maximum
    if model_l1_non_zero > max_nonzeros:
        l1_penalty_min_list.append(l1_penalty)
    
    # Check for L1 penalty minimum
    elif model_l1_non_zero < max_nonzeros:
        l1_penalty_max_list.append(l1_penalty)
    
    print "Non-Zero Parameter: {0}".format(model_l1_limit["coefficients"]["value"].nnz() - 1)
    #print_coefficients(model_l1_limit)
    #model_l1_limit.get("coefficients").print_rows(num_rows = 20)

L1 Penalty: 100000000.0
Non-Zero Parameter: 17
L1 Penalty: 127427498.57
Non-Zero Parameter: 17
L1 Penalty: 162377673.919
Non-Zero Parameter: 17
L1 Penalty: 206913808.111
Non-Zero Parameter: 17
L1 Penalty: 263665089.873
Non-Zero Parameter: 16
L1 Penalty: 335981828.628
Non-Zero Parameter: 16
L1 Penalty: 428133239.872
Non-Zero Parameter: 16
L1 Penalty: 545559478.117
Non-Zero Parameter: 16
L1 Penalty: 695192796.178
Non-Zero Parameter: 16
L1 Penalty: 885866790.41
Non-Zero Parameter: 15
L1 Penalty: 1128837891.68
Non-Zero Parameter: 14
L1 Penalty: 1438449888.29
Non-Zero Parameter: 14
L1 Penalty: 1832980710.83
Non-Zero Parameter: 12
L1 Penalty: 2335721469.09
Non-Zero Parameter: 11
L1 Penalty: 2976351441.63
Non-Zero Parameter: 9
L1 Penalty: 3792690190.73
Non-Zero Parameter: 5
L1 Penalty: 4832930238.57
Non-Zero Parameter: 4
L1 Penalty: 6158482110.66
Non-Zero Parameter: 2
L1 Penalty: 7847599703.51
Non-Zero Parameter: 0
L1 Penalty: 10000000000.0
Non-Zero Parameter: 0


In [19]:
l1_penalty_max = min(l1_penalty_max_list)
l1_penalty_min = max(l1_penalty_min_list)

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [20]:
print "L1 Penalty Maximum: {0}".format(l1_penalty_max)
print "L1 Penalty Minimum: {0}".format(l1_penalty_min)

L1 Penalty Maximum: 3792690190.73
L1 Penalty Minimum: 2976351441.63


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Explore Narrow Range Value For Correct Non-Zero With Lowest RSS For Validation Set

We will now explore the narrow region of `l1_penalty` values we found:

In [79]:
l1_penalty_values = np.linspace(l1_penalty_min, l1_penalty_max, 20)

* for `l1_penalty` in `np.linspace(l1_penalty_min, l1_penalty_max, 20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty = l1_penalty` and `l2_penalty = 0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [21]:
lowest_rss = None
best_l1_penalty = None

for l1_penalty in np.linspace(l1_penalty_min, l1_penalty_max, 20):
    print "L1 Penalty: {0}".format(l1_penalty)
    model_l1_narrow = graphlab.linear_regression.create(training,
                                                        target = "price",
                                                        features = all_features,
                                                        l2_penalty = 0.,
                                                        l1_penalty = l1_penalty,
                                                        validation_set = None,
                                                        verbose = False)
    
    model_l1_non_zero = model_l1_narrow["coefficients"]["value"].nnz() - 1
    
    #model_l1_narrow.get("coefficients").print_rows(num_rows = 20)
    
    # Prediction and RSS
    prediction = model_l1_narrow.predict(validation)
    residual = prediction - validation["price"]
    residual_square = residual ** 2
    rss = residual_square.sum()
    
    # Check lowest RSS
    if model_l1_non_zero == max_nonzeros:
        if lowest_rss is None or rss < lowest_rss:
            lowest_rss = rss
            best_l1_penalty = l1_penalty
            model_l1_narrow.get("coefficients").print_rows(num_rows = 20)
    
    print "Non-Zero: {0}".format(model_l1_non_zero)
    print "RSS: {0}".format(rss)
    
print ""
print "Lowest RSS: {0}".format(lowest_rss)
print "Best L1 Penalty: {0}".format(best_l1_penalty)

L1 Penalty: 2976351441.63
Non-Zero: 9
RSS: 9.66925692362e+14
L1 Penalty: 3019316638.95
Non-Zero: 9
RSS: 9.74019450085e+14
L1 Penalty: 3062281836.27
Non-Zero: 9
RSS: 9.81188367942e+14
L1 Penalty: 3105247033.59
Non-Zero: 9
RSS: 9.89328342459e+14
L1 Penalty: 3148212230.92
Non-Zero: 9
RSS: 9.98783211266e+14
L1 Penalty: 3191177428.24
Non-Zero: 9
RSS: 1.00847716702e+15
L1 Penalty: 3234142625.56
Non-Zero: 9
RSS: 1.01829878055e+15
L1 Penalty: 3277107822.88
Non-Zero: 9
RSS: 1.02824799221e+15
L1 Penalty: 3320073020.2
+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 215235.603644 |  None  |
|     bedrooms     |  None | 1108.36955956 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 16496.2360732 |  None  |
|   sqft_living    |  None | 32.9384118477 |  None  |
| sqft_living_sqrt |  None | 719.868441786 |  None  |
|

`http://datainquisitive.net/wp-content/uploads/2017/01/Lasso_chosingoptimal_L1_penalty-1.html`